##Instalamos librerias

In [1]:
!pip install PyDrive

In [2]:
pip install --upgrade category_encoders

     |████████████████████████████████| 80 kB 3.8 MB/s 


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import category_encoders as ce

from google.colab import files

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
id_train_values = "1RCoa3FWbpcHVC6imhaQQFQEAWNW0qkh0" 
id_test_values = '1xBGPZbt3BYxe1OOYGMJuGnXe_pEqFZ1a'
id_submission_format = "1ei8BhsAAhZpZ8sRqEOVGl12y0JAPdXGV"

downloaded_train_values = drive.CreateFile({'id': id_train_values})
downloaded_test_values = drive.CreateFile({'id': id_test_values})
downloaded_submission_format = drive.CreateFile({'id': id_submission_format})

downloaded_train_values.GetContentFile('train_values.csv')
downloaded_test_values.GetContentFile('test_values.csv')
downloaded_submission_format.GetContentFile('submission_format.csv')

In [6]:
train_values = pd.read_csv('train_values.csv', dtype={
    'building_id': 'int64', 'geo_level_1_id': 'int64',
    'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',
    'count_floors_pre_eq': 'int8', 'age': 'int32',
    'area_percentage': 'int32', 'height_percentage': 'int32',
    'land_surface_condition': 'category', 'foundation_type': 'category',
    'roof_type': 'category', 'ground_floor_type': 'category', 'other_floor_type': 'category',
    'position': 'category', 'plan_configuration': 'category', 'has_superstructure_adobe_mud': 'int8',
    'has_superstructure_mud_mortar_stone': 'int8', 'has_superstructure_stone_flag': 'int8',
    'has_superstructure_cement_mortar_stone': 'int8', 'has_superstructure_mud_mortar_brick': 'int8',
    'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8',
    'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8',
    'has_superstructure_rc_engineered': 'int8', 'has_superstructure_other': 'int8',
    'has_superstructure_other': 'int8', 'legal_ownership_status': 'category',
    'has_secondary_use': 'bool', 'has_secondary_use_agriculture': 'bool',
    'has_secondary_use_hotel': 'bool', 'has_secondary_use_rental': 'bool',
    'has_secondary_use_institution': 'bool', 'has_secondary_use_school': 'bool',
    'has_secondary_use_industry': 'bool', 'has_secondary_use_health_post': 'bool',
    'has_secondary_use_gov_office': 'bool', 'has_secondary_use_use_police': 'bool',
    'has_secondary_use_other': 'bool', 'count_families': 'int8'})

test_values = pd.read_csv('test_values.csv', dtype={
    'building_id': 'int64', 'geo_level_1_id': 'int64',
    'geo_level_2_id': 'int64', 'geo_level_3_id': 'int64',
    'count_floors_pre_eq': 'int8', 'age': 'int32',
    'area_percentage': 'int32', 'height_percentage': 'int32',
    'land_surface_condition': 'category', 'foundation_type': 'category',
    'roof_type': 'category', 'ground_floor_type': 'category', 'other_floor_type': 'category',
    'position': 'category', 'plan_configuration': 'category', 'has_superstructure_adobe_mud': 'int8',
    'has_superstructure_mud_mortar_stone': 'int8', 'has_superstructure_stone_flag': 'int8',
    'has_superstructure_cement_mortar_stone': 'int8', 'has_superstructure_mud_mortar_brick': 'int8',
    'has_superstructure_cement_mortar_brick': 'int8', 'has_superstructure_timber': 'int8',
    'has_superstructure_bamboo': 'int8', 'has_superstructure_rc_non_engineered': 'int8',
    'has_superstructure_rc_engineered': 'int8', 'has_superstructure_other': 'int8',
    'has_superstructure_other': 'int8', 'legal_ownership_status': 'category',
    'has_secondary_use': 'bool', 'has_secondary_use_agriculture': 'bool',
    'has_secondary_use_hotel': 'bool', 'has_secondary_use_rental': 'bool',
    'has_secondary_use_institution': 'bool', 'has_secondary_use_school': 'bool',
    'has_secondary_use_industry': 'bool', 'has_secondary_use_health_post': 'bool',
    'has_secondary_use_gov_office': 'bool', 'has_secondary_use_use_police': 'bool',
    'has_secondary_use_other': 'bool', 'count_families': 'int8'
})

##Creación de Feature Binning Age



In [7]:
train_values['age']

0         30
1         10
2         10
3         10
4         30
          ..
260596    55
260597     0
260598    55
260599    10
260600    10
Name: age, Length: 260601, dtype: int32

In [8]:
#Con esta función hago el binning
cut_labels_4 = ['1', '2', '3', '4','5']
cut_bins = [-1, 20, 40, 60, 80,9999]
train_values['binning_age'] = pd.cut(train_values['age'], bins=cut_bins, labels=cut_labels_4)

In [9]:
train_values = train_values[train_values['age'] != 995]

##Creación de Volume_Perecentage

In [10]:
train_values['volume_percentage'] = train_values['height_percentage'] * train_values['area_percentage']

##Encodeando features con Binary Encoding

In [11]:
train_values.drop(columns=['building_id'], inplace=True)

In [12]:
encoder= ce.BinaryEncoder(cols=['land_surface_condition','foundation_type','roof_type','ground_floor_type','position',"other_floor_type"],return_df=True)

In [13]:
data_encoded=encoder.fit_transform(train_values) 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [14]:
data_encoded

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition_0,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_0,roof_type_1,roof_type_2,ground_floor_type_0,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,other_floor_type_0,other_floor_type_1,other_floor_type_2,position_0,position_1,position_2,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,binning_age,volume_percentage
0,6,487,12198,2,30,6,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,1,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,2,30
1,8,900,2812,2,10,8,7,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,56
2,21,363,8973,2,10,5,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,25
3,22,418,10694,2,10,6,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,d,0,1,0,0,0,0,1,1,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,30
4,11,131,1488,3,30,8,9,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,d,1,0,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,2,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,1,1,0,1,0,q,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,3,18
260597,17,715,2060,2,0,6,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,30
260598,17,51,8163,3,55,6,7,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,3,42
260599,26,39,1851,2,10,14,6,0,0,1,0,0,0,1,0,1,1,0,0,1,1,1,0,0,0,1,1,d,0,0,0,0,0,1,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,84


In [15]:
features = data_encoded[['geo_level_1_id',
    'geo_level_2_id', 'geo_level_3_id','area_percentage','age',"height_percentage",'has_superstructure_adobe_mud',
    'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber','has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
    'has_superstructure_rc_engineered', 'has_superstructure_other',	"land_surface_condition_1",	"land_surface_condition_2",
    	"foundation_type_0",	"foundation_type_1",	"foundation_type_2",	"foundation_type_3","roof_type_1",	"roof_type_2"	,
      "ground_floor_type_1",	"ground_floor_type_2",	"ground_floor_type_3",	
      		"position_1",	"position_2","other_floor_type_0",	"other_floor_type_1",	"other_floor_type_2","binning_age","volume_percentage"]]

In [16]:
features

,geo_level_1_id,geo_level_2_id,geo_level_3_id,area_percentage,age,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_1,roof_type_2,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,position_1,position_2,other_floor_type_0,other_floor_type_1,other_floor_type_2,binning_age,volume_percentage
0,6,487,12198,6,30,5,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,2,30
1,8,900,2812,8,10,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,1,56
2,21,363,8973,5,10,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,25
3,22,418,10694,6,10,5,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,30
4,11,131,1488,8,30,9,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,2,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,6,55,3,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,1,1,3,18
260597,17,715,2060,6,0,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,30
260598,17,51,8163,6,55,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,42
260599,26,39,1851,14,10,6,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,84


##Pre porcesamiento

In [17]:
test_values

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,True,True,False,False,False,False,False,False,False,False,False
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,True,False,False,True,False,False,False,False,False,False,False
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,310028,4,605,3623,3,70,20,6,t,r,q,f,q,t,d,0,1,0,0,0,0,1,0,0,0,0,w,1,True,True,False,False,False,False,False,False,False,False,False
86864,663567,10,1407,11907,3,25,6,7,n,r,n,f,q,s,d,1,1,1,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False
86865,1049160,22,1136,7712,1,50,3,3,t,r,n,f,j,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False
86866,442785,6,1041,912,2,5,9,5,t,r,n,f,q,s,d,1,1,0,0,0,0,0,0,0,0,0,a,1,False,False,False,False,False,False,False,False,False,False,False


In [18]:
#Con esta función hago el binning
cut_labels_4 = ['1', '2', '3', '4','5']
cut_bins = [-1, 20, 40, 60, 80,9999]
test_values['binning_age'] = pd.cut(test_values['age'], bins=cut_bins, labels=cut_labels_4)

In [19]:
test_values = test_values[test_values['age'] != 995]

In [20]:
test_values['volume_percentage'] = test_values['height_percentage'] * test_values['area_percentage']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
test_values.drop(columns=['building_id'], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
encoder_test_values= ce.BinaryEncoder(cols=['land_surface_condition','foundation_type','roof_type','ground_floor_type','position',"other_floor_type"],return_df=True)

In [23]:
test_values_encoded=encoder.fit_transform(test_values) 

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [24]:
test_values_encoded

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition_0,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_0,roof_type_1,roof_type_2,ground_floor_type_0,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,other_floor_type_0,other_floor_type_1,other_floor_type_2,position_0,position_1,position_2,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,binning_age,volume_percentage
0,17,596,11307,3,20,7,6,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,42
1,6,141,11987,2,25,13,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,0,1,0,0,0,0,0,0,0,0,0,v,1,True,True,False,False,False,False,False,False,False,False,False,2,65
2,22,19,10044,2,5,4,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,20
3,26,39,633,1,0,19,3,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,d,0,0,0,0,0,1,0,0,0,0,0,v,2,True,False,False,True,False,False,False,False,False,False,False,1,57
4,17,289,7970,3,15,8,7,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,d,0,1,0,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,1,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,4,605,3623,3,70,20,6,0,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,1,0,d,0,1,0,0,0,0,1,0,0,0,0,w,1,True,True,False,False,False,False,False,False,False,False,False,4,120
86864,10,1407,11907,3,25,6,7,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,1,1,1,0,0,0,0,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,2,42
86865,22,1136,7712,1,50,3,3,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,d,0,1,0,0,0,0,1,0,0,0,0,v,1,False,False,False,False,False,False,False,False,False,False,False,3,9
86866,6,1041,912,2,5,9,5,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,d,1,1,0,0,0,0,0,0,0,0,0,a,1,False,False,False,False,False,False,False,False,False,False,False,1,45


In [25]:
test_features = data_encoded[['geo_level_1_id',
    'geo_level_2_id', 'geo_level_3_id','area_percentage','age',"height_percentage",'has_superstructure_adobe_mud',
    'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag','has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick',
    'has_superstructure_cement_mortar_brick', 'has_superstructure_timber','has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
    'has_superstructure_rc_engineered', 'has_superstructure_other',	"land_surface_condition_1",	"land_surface_condition_2",
    	"foundation_type_0",	"foundation_type_1",	"foundation_type_2",	"foundation_type_3","roof_type_1",	"roof_type_2"	,
      "ground_floor_type_1",	"ground_floor_type_2",	"ground_floor_type_3",	
      		"position_1",	"position_2","other_floor_type_0",	"other_floor_type_1",	"other_floor_type_2","binning_age","volume_percentage"]]

In [26]:
test_features

,geo_level_1_id,geo_level_2_id,geo_level_3_id,area_percentage,age,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,land_surface_condition_1,land_surface_condition_2,foundation_type_0,foundation_type_1,foundation_type_2,foundation_type_3,roof_type_1,roof_type_2,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,position_1,position_2,other_floor_type_0,other_floor_type_1,other_floor_type_2,binning_age,volume_percentage
0,6,487,12198,6,30,5,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,1,2,30
1,8,900,2812,8,10,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,1,56
2,21,363,8973,5,10,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,25
3,22,418,10694,6,10,5,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,30
4,11,131,1488,8,30,9,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,1,0,2,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,6,55,3,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,1,0,0,1,1,3,18
260597,17,715,2060,6,0,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,1,30
260598,17,51,8163,6,55,7,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,42
260599,26,39,1851,14,10,6,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1,0,0,1,84


##Hacemos los archivos con los features definitivos con los que vamos a trabajar con RF

In [27]:
my_submission = pd.DataFrame(data = features)

In [28]:
my_submission.to_csv('featuresFinalesRF.csv')

In [29]:
files.download('featuresFinalesRF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
my_submission_2 = pd.DataFrame(data = test_features)

In [31]:
my_submission.to_csv('featuresFinalesTestRF.csv')

In [32]:
files.download('featuresFinalesTestRF.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>